In [5]:
#  Loading Required Libraries and Data

import pandas as pd
from IPython.display import display

# file path
file_path = "../data/Common Data Warehouse-Orginal.xlsx"


xls = pd.ExcelFile(file_path)


financial_df = xls.parse("Financial")  # Contains transaction amounts
gl_df = xls.parse("GL")  # Contains GL Account details (to identify off-balance transactions)
customer_df = xls.parse("Customer")  # Contains sector information


# Checking unique values in "GL Account Name" to find the correct off-balance label
if "GL Account Name" in gl_df.columns:
    unique_gl_names = gl_df["GL Account Name"].unique()
    print("Unique GL Account Names:", unique_gl_names)
else:
    raise KeyError("Column 'GL Account Name' not found in GL Table")

# Identifying possible Off-Balance labels 
possible_off_balance_labels = [name for name in unique_gl_names if "off" in str(name).lower()]
print("Possible Off-Balance Identifiers:", possible_off_balance_labels)

off_balance_label = possible_off_balance_labels[0] if possible_off_balance_labels else "Off Balance"
print(f"Using Off-Balance Identifier: {off_balance_label}")

# Merging Financial Data with GL Table to Identify Off-Balance Transactions 

financial_gl_df = financial_df.merge(gl_df, left_on="GL Acct ID", right_on="GL Account ID", how="left")

off_balance_count = (financial_gl_df["GL Account Name"] == off_balance_label).sum()
print(f"Total Off-Balance Transactions Found: {off_balance_count}")

#  Processing Data Only If Off-Balance Transactions Exist 

if off_balance_count > 0:
    # Filtering only Off-Balance transactions
    off_balance_df = financial_gl_df[financial_gl_df["GL Account Name"] == off_balance_label]

    # Merging with Customer Table to get Sector Name
    off_balance_sector_df = off_balance_df.merge(customer_df, left_on="Cust ID", right_on="Customer ID", how="left")

    
    sector_off_balance_exposure = off_balance_sector_df.groupby("Sector Name").agg(
        Total_Off_Balance_Exposure=("Amount", "sum")
    )

    # Sorting by highest off-balance exposure
    sector_off_balance_exposure = sector_off_balance_exposure.sort_values(by="Total_Off_Balance_Exposure", ascending=False)

    #  Results
    print("🔹 Sector with Highest Off-Balance Sheet Exposure:")
    display(sector_off_balance_exposure)

    # Save Results to Excel
    output_file_path = "Sector_Off_Balance_Exposure.xlsx"
    sector_off_balance_exposure.to_excel(output_file_path, index=False)

    print(f"Results saved successfully to: {output_file_path}")

else:
    print("No Off-Balance transactions found.")


Unique GL Account Names: ['Off-balance-sheet items' 'Assets']
Possible Off-Balance Identifiers: ['Off-balance-sheet items']
Using Off-Balance Identifier: Off-balance-sheet items
Total Off-Balance Transactions Found: 1318
🔹 Sector with Highest Off-Balance Sheet Exposure:


,Total_Off_Balance_Exposure
Sector Name,
Households,-1.151603e+07
General governments,-8.674180e+08
Other financial corporations,-1.427112e+09
Non financial corporations,-2.233238e+10
Credit institutions,-6.201119e+10


Results saved successfully to: Sector_Off_Balance_Exposure.xlsx
